# Mistral 7B deployment guide
In this tutorial, you will use LMI container from DLC to SageMaker and run inference with it.

Please make sure the following permission granted before running the notebook:

- S3 bucket push access
- SageMaker access

## Step 1: Let's bump up SageMaker and import stuff

In [26]:
%pip install sagemaker --upgrade  --quiet

Note: you may need to restart the kernel to use updated packages.


In [5]:
import boto3
import sagemaker
from sagemaker import Model, image_uris, serializers, deserializers

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


## Step 2: Start preparing model artifacts
In LMI contianer, we expect some artifacts to help setting up the model
- serving.properties (required): Defines the model server settings
- model.py (optional): A python file to define the core inference logic
- requirements.txt (optional): Any additional pip wheel need to install

In [6]:
%%writefile serving.properties
engine=Python
option.model_id=mistralai/Mistral-7B-Instruct-v0.2
option.dtype=fp16
option.max_model_len=8096
option.task=text-generation
option.rolling_batch=vllm
option.tensor_parallel_degree=1
option.device_map=auto

Writing serving.properties


In [7]:
%%sh
mkdir mymodel
mv serving.properties mymodel/
tar czvf mymodel.tar.gz mymodel/
rm -rf mymodel

mymodel/
mymodel/serving.properties


## Step 3: Start building SageMaker endpoint
In this step, we will build SageMaker endpoint from scratch

### Getting the container image URI

[Large Model Inference available DLC](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#large-model-inference-containers)


In [8]:
image_uri = image_uris.retrieve(
        framework="djl-deepspeed",
        region=sess.boto_session.region_name,
        version="0.26.0"
    )

### Upload artifact on S3 and create SageMaker model

In [9]:
s3_code_prefix = "large-model-lmi/code"
bucket = sess.default_bucket()  # bucket to house artifacts
code_artifact = sess.upload_data("mymodel.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {code_artifact}")

model = Model(image_uri=image_uri, model_data=code_artifact, role=role)

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-west-2-376678947624/large-model-lmi/code/mymodel.tar.gz


### 4.2 Create SageMaker endpoint

You need to specify the instance to use and endpoint names

In [10]:
instance_type = "ml.g5.2xlarge"
endpoint_name = sagemaker.utils.name_from_base("lmi-model")

model.deploy(initial_instance_count=1,
             instance_type=instance_type,
             endpoint_name=endpoint_name
            )

# our requests and responses will be in json format so we specify the serializer and the deserializer
predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    serializer=serializers.JSONSerializer(),
)

-----------------!

## Step 5: Test and benchmark the inference

Firstly let's try to run with a wrong inputs

In [18]:
%%time
response = predictor.predict(
    {"inputs": "write a blog about foundation model ops", "parameters": {"max_tokens":2000}}
)

CPU times: user 21.9 ms, sys: 366 µs, total: 22.2 ms
Wall time: 23.6 s


In [19]:
import json
output = json.loads(response)['generated_text']
output

'\n\nTitle: Revolutionizing AI Operations with Foundation Model Ops\n\nIntroduction:\nArtificial Intelligence (AI) has become a game-changer in many industries, enabling businesses to streamline their processes, gain new insights, and enhance customer experiences. However, as AI models become more complex, managing them effectively becomes increasingly challenging. Foundation Model Ops is an emerging approach to managing AI operations that aims to simplify and streamline AI model deployment, scaling, and maintenance. In this blog post, we will explore what Foundation Model Ops is, its benefits, and how it revolutionizes AI operations.\n\nDefinition of Foundation Model Ops:\nFoundation Model Ops refers to the set of practices, tools, and infrastructure that enable organizations to deploy, manage, and operate large-scale AI models efficiently and effectively. These models are called foundation models because they serve as the foundation upon which more specialized models can be built. Fo

In [20]:
2000/23.6

84.7457627118644

## Clean up the environment

In [ ]:
sess.delete_endpoint(endpoint_name)
sess.delete_endpoint_config(endpoint_name)
model.delete_model()